In [21]:
import pandas as pd
import os
import numpy as np
import scipy.sparse as sp
from tqdm.notebook import tqdm

dir_ = '../data'
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [22]:
lh = pd.read_pickle(os.path.join(dir_, folder, file_name))
train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name))

In [23]:
unique_users = len(lh['uid'].unique())
unique_tracks = len(lh['tid'].unique())
tid_start = unique_users
tag_start = unique_users + unique_tracks
pop_start = unique_users + unique_tracks + 19
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))
print ('Times of tracks being listened: ' + str(len(lh)))
lh = None

Number of unique users : 218
Number of unique tracks : 39137
Times of tracks being listened: 453347


In [24]:
len(train), len(test)

(362677, 90670)

In [4]:
# # tag
# l = unique_users + unique_tracks + 19
# i = 0
# sp_rows = []
# sp_cols = []
# for index, row in tqdm(train.iterrows(), total=len(train)):
#     sp_rows.append(i)
#     sp_cols.append(row['uid'])
    
#     sp_rows.append(i)
#     sp_cols.append(tid_start + row['tid'])
    
#     # only one tag
#     sp_rows.append(i)
#     sp_cols.append(tag_start + row['tags'])

#     # many tags
# #     for tag in row['tags']:
# #         sp_rows.append(i)
# #         sp_cols.append(tag_start + tag)
        
#     i += 1
    
# w = np.ones(len(sp_rows))
# train_sp = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(train), l), dtype=float)

In [5]:
# # pop
# l = unique_users + unique_tracks + 1
# i = 0
# v = []
# sp_rows = []
# sp_cols = []
# for index, row in tqdm(train.iterrows(), total=len(train)):
#     sp_rows.append(i)
#     sp_cols.append(row['uid'])
#     v.append(1)
    
#     sp_rows.append(i)
#     sp_cols.append(tid_start + row['tid'])
#     v.append(1)

#     sp_rows.append(i)
#     sp_cols.append(tag_start)
#     v.append(row['pop'])
        
#     i += 1

# train_sp = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(train), l), dtype=float)

In [6]:
# tag and pop
l = unique_users + unique_tracks + 19 + 1
i = 0
v = []
sp_rows = []
sp_cols = []
for index, row in tqdm(train.iterrows(), total=len(train)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tag_start + row['tags'])
    v.append(1)

    sp_rows.append(i)
    sp_cols.append(pop_start)
    v.append(row['pop'])
        
    i += 1

train_sp = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(train), l), dtype=float)

In [7]:
max(sp_rows), max(sp_cols), len(train)

(362676, 39374, 362677)

In [8]:
train_sp.shape

(362677, 39375)

In [9]:
print(train_sp[3,:])

  (0, 61)	1.0
  (0, 2630)	1.0
  (0, 39372)	1.0
  (0, 39374)	11.0


In [10]:
sp.save_npz(os.path.join(dir_, folder,  'train_x_'+ file_name), train_sp)

In [11]:
train_y = []
for index, row in tqdm(train.iterrows(), total=len(train)):
    train_y.append(row['rating'])

In [12]:
np.savetxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), train_y, delimiter=",")

In [13]:
# # tag
# i = 0
# sp_rows = []
# sp_cols = []
# for index, row in tqdm(test.iterrows(), total=len(test)):
#     sp_rows.append(i)
#     sp_cols.append(row['uid'])
    
#     sp_rows.append(i)
#     sp_cols.append(tid_start + row['tid'])
    
#     sp_rows.append(i)
#     sp_cols.append(tag_start + row['tags'])
    
# #     for tag in row['tags']:
# #         sp_rows.append(i)
# #         sp_cols.append(tag_start + tag)

#     i += 1
    
# w = np.ones(len(sp_rows))
# test_sp = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(test), l), dtype=float)

In [14]:
# # pop
# i = 0
# v = []
# sp_rows = []
# sp_cols = []
# for index, row in tqdm(test.iterrows(), total=len(test)):
#     sp_rows.append(i)
#     sp_cols.append(row['uid'])
#     v.append(1)
    
#     sp_rows.append(i)
#     sp_cols.append(tid_start + row['tid'])
#     v.append(1)
    
#     sp_rows.append(i)
#     sp_cols.append(tag_start)
#     v.append(row['pop'])

#     i += 1

# test_sp = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(test), l), dtype=float)

In [25]:
# tag and pop
i = 0
v = []
sp_rows = []
sp_cols = []
for index, row in tqdm(test.iterrows(), total=len(test)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tag_start + row['tags'])
    v.append(1)

    sp_rows.append(i)
    sp_cols.append(pop_start)
    v.append(row['pop'])
        
    i += 1

test_sp = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(test), l), dtype=float)

In [26]:
print(test_sp[3,:])

  (0, 74)	1.0
  (0, 30462)	1.0
  (0, 39356)	1.0
  (0, 39374)	5.0


In [27]:
sp.save_npz(os.path.join(dir_, folder,  'test_x_'+ file_name), test_sp)

In [19]:
test_y = []
for index, row in tqdm(test.iterrows(), total=len(test)):
    test_y.append(row['rating'])

In [20]:
np.savetxt(os.path.join(dir_, folder, 'test_y_' + file_name[:-3] + 'csv'), test_y, delimiter=",")